In [1]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [7]:
input_feats = torch.rand([1,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[8.6582899e-02 7.1769017e-01 2.9701972e-01 7.9147923e-01 1.5550852e-04]
 [6.2197953e-01 3.4608144e-01 4.2699003e-01 7.5858295e-02 7.9285544e-01]]


In [11]:
import torch.nn.functional as F

F.softmax(input_feats, dim = 2)

tensor([[[0.1417, 0.2664, 0.1749, 0.2868, 0.1300],
         [0.2300, 0.1746, 0.1893, 0.1332, 0.2729]]], device='cuda:0')

In [3]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

model_ = BlockSave(model.return_layers(), "fullecapa", "ECAPAweights")
model_.save()

initialblock = BlockSave(model.blocks[0].return_layers(), "initialblock", "ECAPAweights")
initialblock.save()

seres2_1 = BlockSave(model.blocks[1].return_layers(), "seres2_1", "ECAPAweights")
seres2_1.save()
seres2_2 = BlockSave(model.blocks[2].return_layers(), "seres2_2", "ECAPAweights")
seres2_2.save()
seres2_3 = BlockSave(model.blocks[3].return_layers(), "seres2_3", "ECAPAweights")
seres2_3.save()

mfa = BlockSave(model.mfa.return_layers(), "mfa", "ECAPAweights")
mfa.save()

asp = BlockSave(model.asp.return_layers(), "asp", "ECAPAweights")
asp.save()

# asp_bn = BlockSave(model.asp_bn.return_layers(), "asp_bn", "ECAPAweights")
# asp_bn.save()

fc = BlockSave([model.final[1]], "fc", "ECAPAweights")
fc.save()

In [4]:
model.asp(input_feats)

tensor([[[ 0.2879,  0.2103, -0.1322,  ...,  0.2579,  0.1210,  0.2716],
         [ 0.3241,  0.3321,  0.1543,  ...,  0.4876,  0.4087,  0.1353],
         [-0.1158,  0.0232, -0.1112,  ...,  0.2624, -0.3457,  0.3734],
         ...,
         [-0.2016, -0.2271, -0.0020,  ..., -0.2858, -0.2350,  0.3585],
         [-0.1578, -0.2103, -0.0109,  ...,  0.0560, -0.5549, -0.1462],
         [-0.5445,  0.2431,  0.4866,  ..., -0.0935,  0.0419, -0.0091]]],
       device='cuda:0', grad_fn=<MaskedFillBackward0>)
tensor([[[0.0208, 0.0192, 0.0136,  ..., 0.0202, 0.0176, 0.0204],
         [0.0195, 0.0197, 0.0165,  ..., 0.0230, 0.0212, 0.0161],
         [0.0126, 0.0145, 0.0127,  ..., 0.0184, 0.0100, 0.0206],
         ...,
         [0.0129, 0.0126, 0.0157,  ..., 0.0118, 0.0125, 0.0226],
         [0.0140, 0.0133, 0.0163,  ..., 0.0174, 0.0094, 0.0142],
         [0.0091, 0.0200, 0.0256,  ..., 0.0143, 0.0164, 0.0156]]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


tensor([[[0.4871],
         [0.5331],
         [0.4985],
         [0.4988],
         [0.4724],
         [0.6055],
         [0.4554],
         [0.4931],
         [0.5001],
         [0.3983],
         [0.4776],
         [0.6082],
         [0.5276],
         [0.5281],
         [0.4740],
         [0.4087],
         [0.2977],
         [0.3070],
         [0.2811],
         [0.3147],
         [0.2863],
         [0.2687],
         [0.2477],
         [0.2496],
         [0.2593],
         [0.2683],
         [0.2836],
         [0.2639],
         [0.3006],
         [0.2704],
         [0.2908],
         [0.2660]]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [5]:
input_feats.mean(dim=2, keepdim=True).shape

torch.Size([1, 16, 1])

In [6]:
import math

def get_padding_elem(L_in: int, stride: int, kernel_size: int, dilation: int):
    """This function computes the number of elements to add for zero-padding.

    Arguments
    ---------
    L_in : int
    stride: int
    kernel_size : int
    dilation : int

    Returns
    -------
    padding : int
        The size of the padding to be added
    """
    if stride > 1:
        padding = [math.floor(kernel_size / 2), math.floor(kernel_size / 2)]

    else:
        L_out = (
            math.floor((L_in - dilation * (kernel_size - 1) - 1) / stride) + 1
        )
        padding = [
            math.floor((L_in - L_out) / 2),
            math.floor((L_in - L_out) / 2),
        ]
    return padding

get_padding_elem(64,1,5,1)

[2, 2]